In [ ]:
!pip install transformers[torch] sentencepiece PyPDF2 scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.7 MB/s eta 0:00:00


In [ ]:
from google.colab import files

uploaded = files.upload()

filename = list(uploaded.keys())[0]  # Get the filename of the uploaded file

IndexError: list index out of range

In [ ]:
!pip install transformers[torch] sentencepiece PyPDF2 scikit-learn
import PyPDF2

# Open the PDF file in read binary mode ('rb')
with open(filename, 'rb') as pdf_file:
    reader = PyPDF2.PdfReader(pdf_file)
    text = ""
    # Use len(reader.pages) instead of reader.numPages
    for page_num in range(len(reader.pages)):
        # Access pages using reader.pages instead of reader.Pages
        page = reader.pages[page_num]
        text += page.extract_text() # Use extract_text() instead of extractText()

ERROR: Operation cancelled by user


NameError: name 'filename' is not defined

In [ ]:
import re

text = text.lower()  # Convert to lowercase
text = re.sub(r"[^a-zA-Z0-9 ]", "", text)  # Remove punctuation and special characters

In [ ]:
from transformers import AutoTokenizer, AutoModel
import numpy as np
from sklearn.cluster import KMeans

# Load Legal-BERT
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")

# Generate embeddings for sentences
sentences = re.split(r'\n|\. ', text)
sentences = [s for s in sentences if s.strip()]  # Remove empty sentences
embeddings = []
for sentence in sentences:
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    embeddings.append(outputs.last_hidden_state.mean(dim=1).detach().numpy())

# Clustering
num_clusters = min(20, len(sentences))  # Adjust num_clusters if necessary
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(np.vstack(embeddings))
cluster_labels = kmeans.labels_

# Get strong points from each cluster (considering multiple sentences)
strong_points = []
num_strong_points_per_cluster = 3  # Number of strong points to extract per cluster
for cluster_id in range(num_clusters):
    cluster_sentences = [sentences[i] for i, label in enumerate(cluster_labels) if label == cluster_id]
    # Take up to num_strong_points_per_cluster sentences from the cluster
    strong_points.extend(cluster_sentences[:num_strong_points_per_cluster])

# Ensure at least 10 strong points are extracted
if len(strong_points) < 10:
    # If less than 10 strong points, add more from the largest clusters
    cluster_sizes = {cluster_id: len([label for label in cluster_labels if label == cluster_id])
                     for cluster_id in range(num_clusters)}
    sorted_clusters = sorted(cluster_sizes, key=cluster_sizes.get, reverse=True)  # Sort by cluster size

    for cluster_id in sorted_clusters:
        cluster_sentences = [sentences[i] for i, label in enumerate(cluster_labels) if label == cluster_id]
        # Add more sentences from the cluster until we have at least 10 strong points
        strong_points.extend(cluster_sentences[num_strong_points_per_cluster:10 - len(strong_points)])
        if len(strong_points) >= 10:
            break

# Print strong points
print("Strong Points:")
for i, point in enumerate(strong_points):
    print(f"{i + 1}. {point}")

In [ ]:
import re
from transformers import AutoTokenizer, AutoModel
import numpy as np
from sklearn.cluster import KMeans

# Load Legal-BERT
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")

# Sample legal text
text = """<case file murder of john doecase information case number 2023001 case type murder date of incident february 10 2023 location of incident 123 main st anytown usa victim john doe 30 years old suspect jane smith 25 years old investigation report investigating officer detective james johnson summary of investigation on february 10 2023 at approximately 900 pm police received a 911 call reporting a suspicious death at 123 main st upon arrival officers found the victim john doe deceased preliminary investigation revealed signs of forced entry and struggle further investigation led to the identification of jane smith as the prime suspect evidence collected   fingerprints from the crime scene matching jane smith   dna evidence from the victims clothing matching jane smith   security footage showing jane smith near the crime scene on the night of the incident autopsy report cause of death blunt force trauma to the head manner of death homicide toxicology report negative for illicit substances witness statements witness 1 neighbor reported hearing loud noises coming from the victims residence on the night of the incident witness 2 friend of the victim reported seeing jane smith arguing with the victim earlier that day defendants statement jane smith denied any involvement in the murder claiming she was at a movie theater at the time of the incident charges firstdegree murder jane smith is charged with the premeditated murder of john doe court transcripts trial transcript insert transcript of trial proceedings including testimony from witnesses experts and the defendant>"""

# ... (rest of the code for sentence embedding and clustering remains the same)

# Rule matching with improved context and potential punishments
rule_map = {
    "equality": ("Article 14 - Right to Equality", "Potential punishments: Varies depending on the violation, may include fines, imprisonment, or other remedies."),
    "discrimination": ("Article 15 - Prohibition of Discrimination", "Potential punishments: Similar to Article 14, with specific provisions for discrimination based on caste, sex, religion, etc."),
    "freedom": ("Article 19 - Protection of Certain Rights", "Potential punishments: Varies depending on the specific right violated, may include fines, imprisonment, or restrictions on activities."),
    "education": ("Article 21A - Right to Education", "Potential punishments: Penalties for denial of education, may include fines or other measures to ensure compliance."),
    "labour": ("Factories Act, 1948", "Potential punishments: Fines, imprisonment, or closure of factories for violations related to working conditions, safety, and child labor."),
    "environment": ("Environment Protection Act, 1986", "Potential punishments: Fines, imprisonment, or closure of polluting industries for environmental damage."),
    "privacy": ("Right to Privacy (SC, 2017)", "Potential punishments: Compensation for violations of privacy, legal action against infringing parties."),
    "child": ("Child Labour (Prohibition and Regulation) Act, 1986", "Potential punishments: Fines, imprisonment for employers engaging in child labor."),
    "women": ("Protection of Women from Domestic Violence Act, 2005", "Potential punishments: Protection orders, monetary relief, and criminal charges for perpetrators of domestic violence."),
    "harassment": ("POSH Act, 2013", "Potential punishments: Disciplinary action, compensation, and criminal charges for workplace sexual harassment.")
}

def match_indian_rule(sentence):
    matched_rules = []
    for keyword, (rule, punishment) in rule_map.items():
        if keyword.lower() in sentence.lower():
            matched_rules.append((rule, punishment))

    if matched_rules:
        rules_str = ", ".join([rule for rule, _ in matched_rules])
        punishments_str = "\n".join([f"  - {punishment}" for _, punishment in matched_rules])
        return rules_str, punishments_str
    else:
        return "No direct Indian law matched", ""

# Output
print("Strong Points with Related Indian Laws and Potential Punishments:\n")
for i, point in enumerate(strong_points):
    related_law, related_punishment = match_indian_rule(point)
    print(f"Strong point {i + 1}...... {point}")
    print(f"   ↳ Possibly related Indian law: {related_law}")
    if related_punishment:
        print(f"   ↳ Potential punishments:\n{related_punishment}\n")
    else:
        print("\n")

In [ ]:
!pip install kagglehub[pandas-datasets] transformers[torch] sentencepiece
!pip install kaggle # install kaggle package
import kagglehub
from kagglehub import KaggleDatasetAdapter
import re
from transformers import AutoTokenizer, AutoModel
import numpy as np
from sklearn.cluster import KMeans
import os # Import the os module

# Load Legal-BERT
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")

# Load the dataset
# Instead of an empty string, provide the full path or URL to the dataset file
# Example: file_path = 'indian-supreme-court-judgments.csv'
# Or a URL from the dataset: file_path = 'https://www.kaggle.com/datasets/vangap/indian-supreme-court-judgments/download?datasetVersionNumber=1'
#file_path = "indian-supreme-court-judgments.csv"

# Download the dataset using KaggleDatasetAdapter if it doesn't exist
dataset_handle = "vangap/indian-supreme-court-judgments"
file_name = "indian-supreme-court-judgments.csv"
file_path = os.path.join(os.getcwd(), file_name)  # Construct the full file path

# **Changes start here**
# 1. Authenticate with Kaggle API
from google.colab import files
uploaded = files.upload() # Upload your kaggle.json file here
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# **Changes end here**

# 2. Download and unzip the dataset directly
if not os.path.exists(file_path):
    !kaggle datasets download -d {dataset_handle} -p {os.getcwd()} --unzip

# Load the dataset using pandas
import pandas as pd
df = pd.read_csv(file_path)

# ... (rest of the code remains the same)

# Assuming 'Judgment_Text' column contains the judgment text
text = df['Judgment_Text'].iloc[0]  # Get text of the first judgment

# ... (rest of the code remains the same)


# Generate sentence embeddings
sentences = re.split(r'\n|\. ', text)
sentences = [s.strip() for s in sentences if s.strip()]
embeddings = []
for sentence in sentences:
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    embeddings.append(outputs.last_hidden_state.mean(dim=1).detach().numpy())

# Clustering
num_clusters = min(20, len(sentences))
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(np.vstack(embeddings))
cluster_labels = kmeans.labels_

# Extract strong points
strong_points = []
num_strong_points_per_cluster = 3
for cluster_id in range(num_clusters):
    cluster_sentences = [sentences[i] for i, label in enumerate(cluster_labels) if label == cluster_id]
    strong_points.extend(cluster_sentences[:num_strong_points_per_cluster])

# Ensure at least 10 strong points
if len(strong_points) < 10:
    cluster_sizes = {cluster_id: len([label for label in cluster_labels if label == cluster_id])
                     for cluster_id in range(num_clusters)}
    sorted_clusters = sorted(cluster_sizes, key=cluster_sizes.get, reverse=True)
    for cluster_id in sorted_clusters:
        cluster_sentences = [sentences[i] for i, label in enumerate(cluster_labels) if label == cluster_id]
        strong_points.extend(cluster_sentences[num_strong_points_per_cluster:10 - len(strong_points)])
        if len(strong_points) >= 10:
            break

# Create rule map from the dataset
rule_map = {}
for index, row in df.iterrows():
    keywords = re.findall(r'\b\w+\b', row['Judgment_Text'].lower())
    for keyword in keywords:
        if keyword not in rule_map:
            rule_map[keyword] = (row['Case_Name'], row['Legal_Action'])

# Function to match rule
def match_indian_rule(sentence):
    matched_rules = []
    for keyword, (rule, action) in rule_map.items():
        if keyword in sentence.lower():
            matched_rules.append((rule, action))
    if matched_rules:
        rules_str = ", ".join([rule for rule, _ in matched_rules])
        actions_str = ", ".join([action for _, action in matched_rules])
        return rules_str, actions_str
    else:
        return "No direct Indian law matched", ""

# Output
print("Strong Points with Related Indian Laws and Legal Actions:\n")
for i, point in enumerate(strong_points):
    related_law, related_action = match_indian_rule(point)
    print(f"Strong point {i + 1}...... {point}")
    print(f"   ↳ Possibly related case: {related_law}")
    print(f"   ↳ Legal action taken: {related_action}\n")

In [ ]:
import dask.dataframe as dd
import zipfile
import os
from transformers import AutoTokenizer, AutoModel
import numpy as np
from sklearn.cluster import KMeans
import re
from google.colab import files

# 1. Load Legal-BERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")

# 2. Upload zip file containing CSV files
uploaded = files.upload()  # Upload the zip file
zip_file_name = list(uploaded.keys())[0]  # Get the uploaded file name
zip_file_path = zip_file_name  # The path to the uploaded zip file

# 3. Extract the ZIP file if it's not already extracted
extract_dir = "extracted_files"  # Directory for extracted files
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

    # Check if the uploaded file is a zip file
    if zipfile.is_zipfile(zip_file_path):
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
    else:
        print(f"Error: {zip_file_path} is not a zip file. Please upload a valid zip file.")
        exit()  # Exit the program if the file is not valid

# 4. Create a Dask DataFrame from the extracted CSV files
csv_files = [os.path.join(extract_dir, f) for f in os.listdir(extract_dir) if f.endswith('.csv')]

# Print the list of CSV files found to check if it's empty
print("CSV files found:", csv_files)

# Check if csv_files is empty and handle the error
if not csv_files:
    print("Error: No CSV files found in the extracted directory.")
    exit()

ddf = dd.read_csv(csv_files,
                  blocksize=None,  # Read the whole file as a single chunk
                  encoding='latin-1')  # Specify encoding if needed

# ... (rest of your code remains the same)

# 5. Extract and combine text from the 'Judgment_Text' column (assuming it exists)
all_text = ddf['Judgment_Text'].astype(str).str.cat(sep=' ').compute()  # Concatenate all text

# 6. Generate embeddings for sentences
sentences = re.split(r'\n|\. ', all_text)  # Split text into sentences
sentences = [s.strip() for s in sentences if s.strip()]  # Clean empty sentences
embeddings = []

for sentence in sentences:
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    embeddings.append(outputs.last_hidden_state.mean(dim=1).detach().numpy())

# 7. Perform KMeans clustering
num_clusters = min(20, len(sentences))  # Set number of clusters based on sentence count
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(np.vstack(embeddings))  # Perform clustering on sentence embeddings
cluster_labels = kmeans.labels_

# 8. Extract strong points (top sentences per cluster)
strong_points = []
num_strong_points_per_cluster = 3  # Select top 3 sentences per cluster

for cluster_id in range(num_clusters):
    cluster_sentences = [sentences[i] for i, label in enumerate(cluster_labels) if label == cluster_id]
    strong_points.extend(cluster_sentences[:num_strong_points_per_cluster])

# 9. Ensure at least 10 strong points are extracted
if len(strong_points) < 10:
    cluster_sizes = {cluster_id: len([label for label in cluster_labels if label == cluster_id])
                     for cluster_id in range(num_clusters)}
    sorted_clusters = sorted(cluster_sizes, key=cluster_sizes.get, reverse=True)
    for cluster_id in sorted_clusters:
        cluster_sentences = [sentences[i] for i, label in enumerate(cluster_labels) if label == cluster_id]
        strong_points.extend(cluster_sentences[num_strong_points_per_cluster:10 - len(strong_points)])
        if len(strong_points) >= 10:
            break

# 10. Print strong points
print("Strong Points:")
for i, point in enumerate(strong_points):
    print(f"{i + 1}. {point}")



In [ ]:
import dask.dataframe as dd
import zipfile
import os
from transformers import AutoTokenizer, AutoModel
import numpy as np
from sklearn.cluster import KMeans
import re
from google.colab import files

# 1. Load Legal-BERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")

# 2. Upload the zip file from your local PC to Colab
uploaded = files.upload()  # This will prompt you to select the zip file from your PC
zip_file_name = list(uploaded.keys())[0]  # Get the uploaded file name
zip_file_path = zip_file_name  # The path to the uploaded zip file in Colab

# 3. Extract the ZIP file if it's not already extracted
extract_dir = "extracted_files"  # Directory for extracted files
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

    # Check if the uploaded file is a zip file
    if zipfile.is_zipfile(zip_file_path):
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
    else:
        print(f"Error: {zip_file_path} is not a zip file. Please upload a valid zip file.")
        exit()  # Exit the program if the file is not valid

# 4. Create a Dask DataFrame from the extracted CSV files
# Recursively search for CSV files within the extracted directory
import glob
csv_files = glob.glob(os.path.join(extract_dir, '**/*.csv'), recursive=True)

# Print the list of CSV files found to check if it's empty
print("CSV files found:", csv_files)

# Check if csv_files is empty and handle the error
if not csv_files:
    print("Error: No CSV files found in the extracted directory. Check if the zip file contains CSV files in subfolders.")
    exit()

ddf = dd.read_csv(csv_files,
                  blocksize=None,  # Read the whole file as a single chunk
                  encoding='latin-1')  # Specify encoding if needed

# ... (rest of your code remains the same)


# 5. Extract and combine text from the 'Judgment_Text' column (assuming it exists)
all_text = ddf['Judgment_Text'].astype(str).str.cat(sep=' ').compute()  # Concatenate all text

# 6. Generate embeddings for sentences
sentences = re.split(r'\n|\. ', all_text)  # Split text into sentences
sentences = [s.strip() for s in sentences if s.strip()]  # Clean empty sentences
embeddings = []

for sentence in sentences:
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    embeddings.append(outputs.last_hidden_state.mean(dim=1).detach().numpy())

# 7. Perform KMeans clustering
num_clusters = min(20, len(sentences))  # Set number of clusters based on sentence count
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(np.vstack(embeddings))  # Perform clustering on sentence embeddings
cluster_labels = kmeans.labels_

# 8. Extract strong points (top sentences per cluster)
strong_points = []
num_strong_points_per_cluster = 3  # Select top 3 sentences per cluster

for cluster_id in range(num_clusters):
    cluster_sentences = [sentences[i] for i, label in enumerate(cluster_labels) if label == cluster_id]
    strong_points.extend(cluster_sentences[:num_strong_points_per_cluster])

# 9. Ensure at least 10 strong points are extracted
if len(strong_points) < 10:
    cluster_sizes = {cluster_id: len([label for label in cluster_labels if label == cluster_id])
                     for cluster_id in range(num_clusters)}
    sorted_clusters = sorted(cluster_sizes, key=cluster_sizes.get, reverse=True)
    for cluster_id in sorted_clusters:
        cluster_sentences = [sentences[i] for i, label in enumerate(cluster_labels) if label == cluster_id]
        strong_points.extend(cluster_sentences[num_strong_points_per_cluster:10 - len(strong_points)])
        if len(strong_points) >= 10:
            break

# 10. Print strong points
print("Strong Points:")
for i, point in enumerate(strong_points):
    print(f"{i + 1}. {point}")

In [ ]:
!pip install transformers[torch] sentencepiece PyPDF2 scikit-learn dask matplotlib graphviz
import dask.dataframe as dd
import os
from transformers import AutoTokenizer, AutoModel
import numpy as np
from sklearn.cluster import KMeans
import re
from google.colab import files
import pandas as pd
import PyPDF2
import matplotlib.pyplot as plt
import graphviz

# 1. Load Legal-BERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")

# 2. Upload the PDF file to Colab
uploaded = files.upload()  # This will prompt you to select the file
pdf_file_name = list(uploaded.keys())[0]  # Get the uploaded file name
pdf_file_path = pdf_file_name  # Update the file path

# 3. Extract text from the PDF file using PyPDF2
with open(pdf_file_path, 'rb') as pdf_file:
    reader = PyPDF2.PdfReader(pdf_file)
    all_text = ""
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        all_text += page.extract_text()

# 4. Preprocess the text (convert to lowercase, remove punctuation)
all_text = all_text.lower()
all_text = re.sub(r"[^a-zA-Z0-9 ]", "", all_text)

# 5. Generate embeddings for sentences
sentences = re.split(r'\n|\. ', all_text)  # Split text into sentences
sentences = [s.strip() for s in sentences if s.strip()]  # Clean empty sentences
embeddings = []

for sentence in sentences:
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    embeddings.append(outputs.last_hidden_state.mean(dim=1).detach().numpy())

# 6. Perform KMeans clustering
num_clusters = min(20, len(sentences))  # Set number of clusters based on sentence count
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(np.vstack(embeddings))  # Perform clustering on sentence embeddings
cluster_labels = kmeans.labels_

# 7. Extract strong points (top sentences per cluster)
strong_points = []
num_strong_points_per_cluster = 3  # Select top 3 sentences per cluster

for cluster_id in range(num_clusters):
    cluster_sentences = [sentences[i] for i, label in enumerate(cluster_labels) if label == cluster_id]
    strong_points.extend(cluster_sentences[:num_strong_points_per_cluster])

# 8. Ensure at least 10 strong points are extracted
if len(strong_points) < 10:
    cluster_sizes = {cluster_id: len([label for label in cluster_labels if label == cluster_id])
                     for cluster_id in range(num_clusters)}
    sorted_clusters = sorted(cluster_sizes, key=cluster_sizes.get, reverse=True)
    for cluster_id in sorted_clusters:
        cluster_sentences = [sentences[i] for i, label in enumerate(cluster_labels) if label == cluster_id]
        strong_points.extend(cluster_sentences[num_strong_points_per_cluster:10 - len(strong_points)])
        if len(strong_points) >= 10:
            break

# 9. Print strong points
print("Strong Points:")
for i, point in enumerate(strong_points):
    print(f"{i + 1}. {point}")

# 10. Generate image of results
plt.figure(figsize=(10, 6))  # Adjust figure size as needed
plt.text(0.05, 0.95, "Strong Points:", fontsize=14, fontweight='bold')
for i, point in enumerate(strong_points):
    plt.text(0.05, 0.9 - (i * 0.05), f"{i + 1}. {point}", fontsize=10)
plt.axis('off')  # Remove axes
plt.savefig("strong_points.png")  # Save the image
plt.show()

# 11. Create diagram of model training (conceptual)
dot = graphviz.Digraph(comment='Legal-BERT Training')
dot.node('A', 'Legal Documents')
dot.node('B', 'Preprocessing')
dot.node('C', 'Legal-BERT Model')
dot.node('D', 'Fine-tuning')
dot.node('E', 'Trained Legal-BERT')

dot.edges(['AB', 'BC', 'CD', 'DE'])  # Define the connections between nodes
dot.render('legal_bert_training', view=True)  # Render and display the diagram

Saving 17684-2001___jonew__judis__26559.pdf to 17684-2001___jonew__judis__26559.pdf
Strong Points:
1. httpjudisnicin supreme court of india page 1 of 14 case noappeal civil  6951 of 2004petitionerjp srivastava  sons pvt ltd  orsrespondentms gwalior sugar co ltd  orsdate of judgment 26102004benchruma pal  arun kumarjudgmentj u d g m e n tarising out of slp no 220442001ruma pal j leave granted        this appeal arises out of proceedings initiated under sections 397 and 398 of the companies act hereinafter referred to as the act by a group of minority shareholders complaining of mismanagement and oppression in respect of the respondent no1 company ms gwalior sugar company ltd referred to as the company  the appellants are the unsuccessful petitioners  the primary question to be resolved in this appeal is whether they held the requisite onetenth of the issued share capital of the company under section 399 1 of the act when they filed the petition under ss 397 and 398        the shares of 

'legal_bert_training.pdf'

In [ ]:
!pip install transformers[torch] sentencepiece PyPDF2 scikit-learn dask reportlab graphviz

import dask.dataframe as dd
import os
from transformers import AutoTokenizer, AutoModel
import numpy as np
from sklearn.cluster import KMeans
import re
from google.colab import files
import pandas as pd
import PyPDF2
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
import graphviz

# 1. Load Legal-BERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")

# 2. Upload the PDF file to Colab
uploaded = files.upload()  # This will prompt you to select the file
pdf_file_name = list(uploaded.keys())[0]  # Get the uploaded file name
pdf_file_path = pdf_file_name  # Update the file path

# 3. Extract text from the PDF file using PyPDF2
with open(pdf_file_path, 'rb') as pdf_file:
    reader = PyPDF2.PdfReader(pdf_file)
    all_text = ""
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        all_text += page.extract_text()

# 4. Preprocess the text (convert to lowercase, remove punctuation)
all_text = all_text.lower()
all_text = re.sub(r"[^a-zA-Z0-9 ]", "", all_text)

# 5. Generate embeddings for sentences
sentences = re.split(r'\n|\. ', all_text)  # Split text into sentences
sentences = [s.strip() for s in sentences if s.strip()]  # Clean empty sentences
embeddings = []

for sentence in sentences:
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    embeddings.append(outputs.last_hidden_state.mean(dim=1).detach().numpy())

# 6. Perform KMeans clustering
num_clusters = min(20, len(sentences))  # Set number of clusters based on sentence count
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(np.vstack(embeddings))  # Perform clustering on sentence embeddings
cluster_labels = kmeans.labels_

# 7. Extract strong points (top sentences per cluster)
strong_points = []
num_strong_points_per_cluster = 5  # Select top 5 sentences per cluster

for cluster_id in range(num_clusters):
    cluster_sentences = [sentences[i] for i, label in enumerate(cluster_labels) if label == cluster_id]
    strong_points.extend(cluster_sentences[:num_strong_points_per_cluster])

# 8. Ensure at least 15 strong points are extracted
if len(strong_points) < 15:
    cluster_sizes = {cluster_id: len([label for label in cluster_labels if label == cluster_id])
                     for cluster_id in range(num_clusters)}
    sorted_clusters = sorted(cluster_sizes, key=cluster_sizes.get, reverse=True)
    for cluster_id in sorted_clusters:
        cluster_sentences = [sentences[i] for i, label in enumerate(cluster_labels) if label == cluster_id]
        strong_points.extend(cluster_sentences[num_strong_points_per_cluster:15 - len(strong_points)])
        if len(strong_points) >= 15:
            break

# 9. Print strong points
print("Strong Points:")
for i, point in enumerate(strong_points):
    print(f"{i + 1}. {point}")

# 10. Generate PDF of results
doc = SimpleDocTemplate("strong_points.pdf", pagesize=letter)
styles = getSampleStyleSheet()
story = [Paragraph("Strong Points:", styles['h1'])]
for i, point in enumerate(strong_points):
    story.append(Paragraph(f"{i + 1}. {point}", styles['Normal']))
    story.append(Spacer(1, 12))  # Add some space between points
doc.build(story)

# 11. Create diagram of model training (conceptual)
dot = graphviz.Digraph(comment='Legal-BERT Training')
dot.node('A', 'Legal Documents')
dot.node('B', 'Preprocessing')
dot.node('C', 'Legal-BERT Model')
dot.node('D', 'Fine-tuning')
dot.node('E', 'Trained Legal-BERT')

dot.edges(['AB', 'BC', 'CD', 'DE'])  # Define the connections between nodes
dot.render('legal_bert_training', view=True)  # Render and display the diagram


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Saving -0___jonew__judis__11344.pdf to -0___jonew__judis__11344.pdf
Strong Points:
1. httpjudisnicin supreme court of india page 1 of 1 petitionerunion of india        vsrespondentrajiv yadavdate of judgment21071994benchkuldip singh jbenchkuldip singh jpunchhi mmramaswamy kcitation 1995 air   14            1994 scc  6  38 jt 1994 5    54        1994 scale  3617actheadnotejudgment                           order1   we have today pronounced judgment in union of india  vrajiv  yadav  ias1   the  appeal  has  been  allowed  theimpugned judgment of the central administrative tribunal hasbeen set aside and the principles of cadre allocation  forreserved candidates have been upheld2   in  view  of our judgment in rajiv yadav  case  1  thisappeal  by  the union of india has to be allowed   for  thereasons  given  and the conclusions reached by us  in  rajivyadav  case1 we allow this appeal set aside  the  impugnedjudgment  of the tribunal dated 27111992 and  dismiss  theapplication  filed by v

'legal_bert_training.pdf'

In [ ]:
!apt-get install poppler-utils
!apt-get install tesseract-ocr
!apt-get install libtesseract-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Fetched 186 kB in 0s (373 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126209 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.6_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.6) ...
Setting up poppler-utils (22.02.0-2ubuntu0.6) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
T

In [ ]:
!apt-get install poppler-utils
!apt-get install tesseract-ocr
!apt-get install libtesseract-dev
!pip install pytesseract pdf2image Pillow gTTS

import pytesseract
from pdf2image import convert_from_path
from google.colab import files
from PIL import Image
from gtts import gTTS
from IPython.display import Audio

# 1. Upload the PDF file
uploaded = files.upload()
pdf_file_path = list(uploaded.keys())[0]  # Get the uploaded PDF file name

# 2. Convert PDF to images (one image per page)
images = convert_from_path(pdf_file_path)

# 3. Run OCR on each page image to extract text
extracted_text = ""
for i, image in enumerate(images):
    # Convert image to text using pytesseract
    text = pytesseract.image_to_string(image)
    extracted_text += f"--- Page {i+1} ---\n{text}\n\n"

# 4. Display the extracted text
print(extracted_text)

# 5. Convert extracted text to speech using gTTS
tts = gTTS(text=extracted_text, lang='en')  # You can change 'en' to other languages
tts.save("extracted_text.mp3")

# 6. Play the audio
Audio("extracted_text.mp3", autoplay=True)

# 7. Optionally display the images of the PDF pages (this step is optional)
# for i, image in enumerate(images):
#     display(image)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


Saving strong_points (1).pdf to strong_points (1).pdf
--- Page 1 ---
Strong Points:

1. httpjudisnicin supreme court of india page 1 of 1 petitionerunion of india vsrespondentrajiv yadavdate
of judgment21071994benchkuldip singh jbenchkuldip singh jpunchhi mmramaswamy kcitation 1995 air
14 1994 scc 6 38 jt 1994 5 54 1994 scale 3617actheadnotejudgment order1 we have today pronounced
judgment in union of india vrajiv yadav ias1 the appeal has been allowed theimpugned judgment of the
central administrative tribunal hasbeen set aside and the principles of cadre allocation forreserved
candidates have been upheld2 in view of our judgment in rajiv yadav case 1 thisappeal by the union of
india has to be allowed for thereasons given and the conclusions reached by us in rajivyadav case1 we
allow this appeal set aside the impugnedjudgment of the tribunal dated 27111992 and dismiss
theapplication filed by vinay kumar before the tribunal nocosts



